In [58]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [74]:
## 모델 불러오기
from tensorflow.python.keras.models import load_model

model = load_model('/content/drive/MyDrive/데이터비즈니스_프로젝트/LSTM/lstm_best_model.h5')
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, None, 45)          307080    
_________________________________________________________________
lstm_7 (LSTM)                (None, 64)                28160     
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 195       
Total params: 335,435
Trainable params: 335,435
Non-trainable params: 0
_________________________________________________________________


In [35]:
import pandas as pd

news_df = pd.read_csv('/content/drive/MyDrive/데이터비즈니스_프로젝트/주가데이터/samsung_news.csv')
news_df=news_df[['title']]

In [36]:
news_df.head()

,title
0,삼성 준법위-사업지원 TF 회동…콘트롤타워 부활 논의했나
1,"삼성전자, 장애인 직원 채용 자회사 만든다…일자리 창출"
2,"삼성전자, 자회사형 장애인 표준사업장 만든다"
3,"한국남동발전-삼성전자, '탄소중립' 함께 달린다"
4,"삼성 임원들, 격려금 100억 모아 헌혈버스 4대 기부"


In [37]:
import pandas as pd
import numpy as np
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [38]:
## 특수문자 제거
news_df['title'] = [re.sub('[^A-Za-z0-9가-힣]', ' ', s) for s in news_df['title']]

# 한국어만 남기고 싶으면
#news_df['title'] = [re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]',' ',s) for s in news_df['title']]

news_df.head()

,title
0,삼성 준법위 사업지원 TF 회동 콘트롤타워 부활 논의했나
1,삼성전자 장애인 직원 채용 자회사 만든다 일자리 창출
2,삼성전자 자회사형 장애인 표준사업장 만든다
3,한국남동발전 삼성전자 탄소중립 함께 달린다
4,삼성 임원들 격려금 100억 모아 헌혈버스 4대 기부


In [10]:
!pip install -v python-mecab-ko

Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.purelib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.headers does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: /usr/include/python3.7m/UNKNOWN
Value for scheme.scripts does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/bin
sysconfig: /usr/bin
Value for scheme.data does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local
sysconfig: /usr
Additional context:
user = False
home

In [39]:
import mecab
mecab = mecab.MeCab()

In [40]:
news_df['token'] = news_df['title'].apply(mecab.morphs)
news_df.head()

,title,token
0,삼성 준법위 사업지원 TF 회동 콘트롤타워 부활 논의했나,"[삼성, 준, 법위, 사업, 지원, TF, 회동, 콘트롤, 타워, 부활, 논의, 했..."
1,삼성전자 장애인 직원 채용 자회사 만든다 일자리 창출,"[삼성전자, 장애, 인, 직원, 채용, 자회사, 만든다, 일자리, 창출]"
2,삼성전자 자회사형 장애인 표준사업장 만든다,"[삼성전자, 자회사, 형, 장애, 인, 표준, 사업장, 만든다]"
3,한국남동발전 삼성전자 탄소중립 함께 달린다,"[한국남, 동발, 전, 삼성전자, 탄소, 중립, 함께, 달린다]"
4,삼성 임원들 격려금 100억 모아 헌혈버스 4대 기부,"[삼성, 임원, 들, 격려금, 100, 억, 모아, 헌혈, 버스, 4, 대, 기부]"


In [41]:
news_df['remove_stopwords'] = None
stopwords = ['있','수','하','기','년','인','되','면','한다','에서','그','했','다','을','의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
for i in range(len(news_df)):
  stopwords_removed_sentence = [word for word in news_df.iloc[i,1] if not word in stopwords] # 불용어 제거
  news_df['remove_stopwords'][i] = stopwords_removed_sentence

In [42]:
input_news = news_df['remove_stopwords']
input_news.head()

0    [삼성, 준, 법위, 사업, 지원, TF, 회동, 콘트롤, 타워, 부활, 논의, 나]
1              [삼성전자, 장애, 직원, 채용, 자회사, 만든다, 일자리, 창출]
2                   [삼성전자, 자회사, 형, 장애, 표준, 사업장, 만든다]
3                [한국남, 동발, 전, 삼성전자, 탄소, 중립, 함께, 달린다]
4        [삼성, 임원, 격려금, 100, 억, 모아, 헌혈, 버스, 4, 대, 기부]
Name: remove_stopwords, dtype: object

In [43]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(input_news)
# 정수 인코딩
input_news_encoded = tokenizer.texts_to_sequences(input_news)
print(input_news_encoded[:5])

[[1, 115, 116, 117, 26, 118, 119, 120, 121, 37, 122, 27], [2, 28, 38, 123, 29, 39, 40, 41], [2, 29, 42, 28, 19, 43, 39], [124, 125, 20, 2, 44, 45, 46, 126], [1, 30, 127, 21, 31, 128, 11, 16, 12, 6, 47]]


In [44]:
# 단어 길이 45으로 padding
max_len = 45
pad_new = pad_sequences(input_news_encoded, maxlen = max_len)
pad_new

array([[  0,   0,   0, ...,  37, 122,  27],
       [  0,   0,   0, ...,  39,  40,  41],
       [  0,   0,   0, ...,  19,  43,  39],
       ...,
       [  0,   0,   0, ..., 338,  71, 339],
       [  0,   0,   0, ..., 345,  24, 346],
       [  0,   0,   0, ..., 350,  15,  94]], dtype=int32)

In [75]:
score = model.predict(pad_new)
score[:5]

array([[0.25211138, 0.524239  , 0.22364964],
       [0.5017537 , 0.38561386, 0.11263245],
       [0.60621506, 0.28752574, 0.10625934],
       [0.6526745 , 0.24108286, 0.10624268],
       [0.5497771 , 0.3251817 , 0.12504117]], dtype=float32)

In [76]:
score[0]

array([0.25211138, 0.524239  , 0.22364964], dtype=float32)

In [77]:
score_1 = score[0]
score_1
#'neutral':0, 'positive':1, 'negative':2
print(news_df.loc[0,'title'])
if score_1.argmax() == 0:
  print("{:.2f}% 확률로 중성 리뷰입니다.\n".format(score_1[0]/sum(score_1) * 100))
elif score_1.argmax() == 1:
  print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score_1[1]/sum(score_1) * 100))
elif score_1.argmax() == 2:
  print("{:.2f}% 확률로 부정 리뷰입니다.\n".format(score_1[2]/sum(score_1) * 100))

삼성 준법위 사업지원 TF 회동 콘트롤타워 부활 논의했나
52.42% 확률로 긍정 리뷰입니다.



In [78]:
# news_df 에 뉴스기사 dataframe
# score 뽑아내기
def preprocessing(news_df):
  #news_df['title'] = [re.sub('[^A-Za-z0-9가-힣]', ' ', s) for s in news_df['title']]
  news_df['token'] = news_df['title'].apply(mecab.morphs)
  input_news = news_df['token']
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(input_news)
  input_news_encoded = tokenizer.texts_to_sequences(input_news)
  max_len = 76
  pad_new = pad_sequences(input_news_encoded, maxlen = max_len)
  score = model.predict(pad_new)
  #'neutral':0, 'positive':1, 'negative':2
  # print(news_df.loc[0,"'title'"])
  # if score_1.argmax() == 0:
  #   print("{:.2f}% 확률로 중성 리뷰입니다.\n".format(score_1[0]/sum(score_1) * 100))
  # elif score_1.argmax() == 1:
  #   print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score_1[1]/sum(score_1) * 100))
  # elif score_1.argmax() == 2:
  #   print("{:.2f}% 확률로 부정 리뷰입니다.\n".format(score_1[2]/sum(score_1) * 100))

In [79]:
news_df['predict'] = None
news_df['prob'] = None
news_df

,title,token,remove_stopwords,predict,prob
0,삼성 준법위 사업지원 TF 회동 콘트롤타워 부활 논의했나,"[삼성, 준, 법위, 사업, 지원, TF, 회동, 콘트롤, 타워, 부활, 논의, 했...","[삼성, 준, 법위, 사업, 지원, TF, 회동, 콘트롤, 타워, 부활, 논의, 나]",None,None
1,삼성전자 장애인 직원 채용 자회사 만든다 일자리 창출,"[삼성전자, 장애, 인, 직원, 채용, 자회사, 만든다, 일자리, 창출]","[삼성전자, 장애, 직원, 채용, 자회사, 만든다, 일자리, 창출]",None,None
2,삼성전자 자회사형 장애인 표준사업장 만든다,"[삼성전자, 자회사, 형, 장애, 인, 표준, 사업장, 만든다]","[삼성전자, 자회사, 형, 장애, 표준, 사업장, 만든다]",None,None
3,한국남동발전 삼성전자 탄소중립 함께 달린다,"[한국남, 동발, 전, 삼성전자, 탄소, 중립, 함께, 달린다]","[한국남, 동발, 전, 삼성전자, 탄소, 중립, 함께, 달린다]",None,None
4,삼성 임원들 격려금 100억 모아 헌혈버스 4대 기부,"[삼성, 임원, 들, 격려금, 100, 억, 모아, 헌혈, 버스, 4, 대, 기부]","[삼성, 임원, 격려금, 100, 억, 모아, 헌혈, 버스, 4, 대, 기부]",None,None
5,삼성전자 찾은 민주당 기업 미 중 패권 희생양 안되도록 할 것,"[삼성전자, 찾, 은, 민주당, 기업, 미, 중, 패권, 희생양, 안, 되, 도록,...","[삼성전자, 찾, 민주당, 기업, 미, 중, 패권, 희생양, 안, 도록, 할, 것]",None,None
6,세계 1등 한국제품 몇개일까 삼성전자가 올킬,"[세계, 1, 등, 한국, 제품, 몇, 개, 일까, 삼성전자, 가, 올, 킬]","[세계, 1, 등, 한국, 제품, 몇, 개, 일까, 삼성전자, 올, 킬]",None,None
7,국내 100대기업 내 여성 임원 400명 돌파 삼성전자 최다,"[국내, 100, 대, 기업, 내, 여성, 임원, 400, 명, 돌파, 삼성전자, 최다]","[국내, 100, 대, 기업, 내, 여성, 임원, 400, 명, 돌파, 삼성전자, 최다]",None,None
8,삼성 대한적십자사에 헌혈버스 4대 기증 이재용 동행 철학 실천,"[삼성, 대한, 적십자사, 에, 헌혈, 버스, 4, 대, 기증, 이재용, 동행, 철...","[삼성, 대한, 적십자사, 헌혈, 버스, 4, 대, 기증, 이재용, 동행, 철학, 실천]",None,None
9,세계 1 2위면 뭐하나 창고에 쌓이는 삼성 LG TV,"[세계, 1, 2, 위, 면, 뭐, 하나, 창고, 에, 쌓이, 는, 삼성, LG, TV]","[세계, 1, 2, 위, 뭐, 하나, 창고, 쌓이, 삼성, LG, TV]",None,None


In [80]:
for i in range(len(news_df)):
  if score[i].argmax() == 0:
    sent_predict = 0
    prob = ("{:.2f}% 확률로 중성 리뷰입니다.\n".format(score[i][0]/sum(score[i]) * 100))
  elif score[i].argmax() == 1:
    sent_predict = 1
    prob = ("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score[i][1]/sum(score[i]) * 100))
  elif score[i].argmax() == 2:
    sent_predict = 2
    prob = ("{:.2f}% 확률로 부정 리뷰입니다.\n".format(score[i][2]/sum(score[i]) * 100))
  news_df['predict'][i] = sent_predict
  news_df['prob'][i] = prob

In [81]:
news_df

,title,token,remove_stopwords,predict,prob
0,삼성 준법위 사업지원 TF 회동 콘트롤타워 부활 논의했나,"[삼성, 준, 법위, 사업, 지원, TF, 회동, 콘트롤, 타워, 부활, 논의, 했...","[삼성, 준, 법위, 사업, 지원, TF, 회동, 콘트롤, 타워, 부활, 논의, 나]",1,52.42% 확률로 긍정 리뷰입니다.\n
1,삼성전자 장애인 직원 채용 자회사 만든다 일자리 창출,"[삼성전자, 장애, 인, 직원, 채용, 자회사, 만든다, 일자리, 창출]","[삼성전자, 장애, 직원, 채용, 자회사, 만든다, 일자리, 창출]",0,50.18% 확률로 중성 리뷰입니다.\n
2,삼성전자 자회사형 장애인 표준사업장 만든다,"[삼성전자, 자회사, 형, 장애, 인, 표준, 사업장, 만든다]","[삼성전자, 자회사, 형, 장애, 표준, 사업장, 만든다]",0,60.62% 확률로 중성 리뷰입니다.\n
3,한국남동발전 삼성전자 탄소중립 함께 달린다,"[한국남, 동발, 전, 삼성전자, 탄소, 중립, 함께, 달린다]","[한국남, 동발, 전, 삼성전자, 탄소, 중립, 함께, 달린다]",0,65.27% 확률로 중성 리뷰입니다.\n
4,삼성 임원들 격려금 100억 모아 헌혈버스 4대 기부,"[삼성, 임원, 들, 격려금, 100, 억, 모아, 헌혈, 버스, 4, 대, 기부]","[삼성, 임원, 격려금, 100, 억, 모아, 헌혈, 버스, 4, 대, 기부]",0,54.98% 확률로 중성 리뷰입니다.\n
5,삼성전자 찾은 민주당 기업 미 중 패권 희생양 안되도록 할 것,"[삼성전자, 찾, 은, 민주당, 기업, 미, 중, 패권, 희생양, 안, 되, 도록,...","[삼성전자, 찾, 민주당, 기업, 미, 중, 패권, 희생양, 안, 도록, 할, 것]",1,45.56% 확률로 긍정 리뷰입니다.\n
6,세계 1등 한국제품 몇개일까 삼성전자가 올킬,"[세계, 1, 등, 한국, 제품, 몇, 개, 일까, 삼성전자, 가, 올, 킬]","[세계, 1, 등, 한국, 제품, 몇, 개, 일까, 삼성전자, 올, 킬]",0,43.78% 확률로 중성 리뷰입니다.\n
7,국내 100대기업 내 여성 임원 400명 돌파 삼성전자 최다,"[국내, 100, 대, 기업, 내, 여성, 임원, 400, 명, 돌파, 삼성전자, 최다]","[국내, 100, 대, 기업, 내, 여성, 임원, 400, 명, 돌파, 삼성전자, 최다]",0,66.47% 확률로 중성 리뷰입니다.\n
8,삼성 대한적십자사에 헌혈버스 4대 기증 이재용 동행 철학 실천,"[삼성, 대한, 적십자사, 에, 헌혈, 버스, 4, 대, 기증, 이재용, 동행, 철...","[삼성, 대한, 적십자사, 헌혈, 버스, 4, 대, 기증, 이재용, 동행, 철학, 실천]",1,44.36% 확률로 긍정 리뷰입니다.\n
9,세계 1 2위면 뭐하나 창고에 쌓이는 삼성 LG TV,"[세계, 1, 2, 위, 면, 뭐, 하나, 창고, 에, 쌓이, 는, 삼성, LG, TV]","[세계, 1, 2, 위, 뭐, 하나, 창고, 쌓이, 삼성, LG, TV]",1,44.22% 확률로 긍정 리뷰입니다.\n


In [82]:
news_df[news_df['predict'] == 0]

,title,token,remove_stopwords,predict,prob
1,삼성전자 장애인 직원 채용 자회사 만든다 일자리 창출,"[삼성전자, 장애, 인, 직원, 채용, 자회사, 만든다, 일자리, 창출]","[삼성전자, 장애, 직원, 채용, 자회사, 만든다, 일자리, 창출]",0,50.18% 확률로 중성 리뷰입니다.\n
2,삼성전자 자회사형 장애인 표준사업장 만든다,"[삼성전자, 자회사, 형, 장애, 인, 표준, 사업장, 만든다]","[삼성전자, 자회사, 형, 장애, 표준, 사업장, 만든다]",0,60.62% 확률로 중성 리뷰입니다.\n
3,한국남동발전 삼성전자 탄소중립 함께 달린다,"[한국남, 동발, 전, 삼성전자, 탄소, 중립, 함께, 달린다]","[한국남, 동발, 전, 삼성전자, 탄소, 중립, 함께, 달린다]",0,65.27% 확률로 중성 리뷰입니다.\n
4,삼성 임원들 격려금 100억 모아 헌혈버스 4대 기부,"[삼성, 임원, 들, 격려금, 100, 억, 모아, 헌혈, 버스, 4, 대, 기부]","[삼성, 임원, 격려금, 100, 억, 모아, 헌혈, 버스, 4, 대, 기부]",0,54.98% 확률로 중성 리뷰입니다.\n
6,세계 1등 한국제품 몇개일까 삼성전자가 올킬,"[세계, 1, 등, 한국, 제품, 몇, 개, 일까, 삼성전자, 가, 올, 킬]","[세계, 1, 등, 한국, 제품, 몇, 개, 일까, 삼성전자, 올, 킬]",0,43.78% 확률로 중성 리뷰입니다.\n
7,국내 100대기업 내 여성 임원 400명 돌파 삼성전자 최다,"[국내, 100, 대, 기업, 내, 여성, 임원, 400, 명, 돌파, 삼성전자, 최다]","[국내, 100, 대, 기업, 내, 여성, 임원, 400, 명, 돌파, 삼성전자, 최다]",0,66.47% 확률로 중성 리뷰입니다.\n
10,삼성 저격 삼성생명법 2년 만의 부활,"[삼성, 저격, 삼성생명, 법, 2, 년, 만, 의, 부활]","[삼성, 저격, 삼성생명, 법, 2, 만, 부활]",0,42.39% 확률로 중성 리뷰입니다.\n
11,세계에서 팔린 TV 3대 중 1대는 삼성 누적 점유율 1위,"[세계, 에서, 팔린, TV, 3, 대, 중, 1, 대, 는, 삼성, 누적, 점유,...","[세계, 팔린, TV, 3, 대, 중, 1, 대, 삼성, 누적, 점유, 율, 1, 위]",0,74.09% 확률로 중성 리뷰입니다.\n
12,삼성전자 e식품관 100만 방문 기념 고객 감사 기획전,"[삼성전자, e, 식품, 관, 100, 만, 방문, 기념, 고객, 감사, 기획, 전]","[삼성전자, e, 식품, 관, 100, 만, 방문, 기념, 고객, 감사, 기획, 전]",0,75.93% 확률로 중성 리뷰입니다.\n
13,품질경영대회 은탑훈장에 TV 16년 1위 삼성전자 김형남 부사장,"[품질, 경영, 대회, 은, 탑, 훈장, 에, TV, 16, 년, 1, 위, 삼성전...","[품질, 경영, 대회, 탑, 훈장, TV, 16, 1, 위, 삼성전자, 김형남, 부사장]",0,92.41% 확률로 중성 리뷰입니다.\n


In [83]:
news_df[news_df['predict'] == 1]

,title,token,remove_stopwords,predict,prob
0,삼성 준법위 사업지원 TF 회동 콘트롤타워 부활 논의했나,"[삼성, 준, 법위, 사업, 지원, TF, 회동, 콘트롤, 타워, 부활, 논의, 했...","[삼성, 준, 법위, 사업, 지원, TF, 회동, 콘트롤, 타워, 부활, 논의, 나]",1,52.42% 확률로 긍정 리뷰입니다.\n
5,삼성전자 찾은 민주당 기업 미 중 패권 희생양 안되도록 할 것,"[삼성전자, 찾, 은, 민주당, 기업, 미, 중, 패권, 희생양, 안, 되, 도록,...","[삼성전자, 찾, 민주당, 기업, 미, 중, 패권, 희생양, 안, 도록, 할, 것]",1,45.56% 확률로 긍정 리뷰입니다.\n
8,삼성 대한적십자사에 헌혈버스 4대 기증 이재용 동행 철학 실천,"[삼성, 대한, 적십자사, 에, 헌혈, 버스, 4, 대, 기증, 이재용, 동행, 철...","[삼성, 대한, 적십자사, 헌혈, 버스, 4, 대, 기증, 이재용, 동행, 철학, 실천]",1,44.36% 확률로 긍정 리뷰입니다.\n
9,세계 1 2위면 뭐하나 창고에 쌓이는 삼성 LG TV,"[세계, 1, 2, 위, 면, 뭐, 하나, 창고, 에, 쌓이, 는, 삼성, LG, TV]","[세계, 1, 2, 위, 뭐, 하나, 창고, 쌓이, 삼성, LG, TV]",1,44.22% 확률로 긍정 리뷰입니다.\n
28,2년 더 일했는데 신입 연봉이랑 같다니 삼성 직원의 분노,"[2, 년, 더, 일, 했, 는데, 신입, 연봉, 이랑, 같, 다니, 삼성, 직원,...","[2, 더, 일, 는데, 신입, 연봉, 이랑, 같, 다니, 삼성, 직원, 분노]",1,55.63% 확률로 긍정 리뷰입니다.\n
48,삼성 대한적십자사에 헌혈버스 4대 전달 10년간 40대 지원 예정,"[삼성, 대한, 적십자사, 에, 헌혈, 버스, 4, 대, 전달, 10, 년, 간, ...","[삼성, 대한, 적십자사, 헌혈, 버스, 4, 대, 전달, 10, 간, 40, 대,...",1,43.91% 확률로 긍정 리뷰입니다.\n
50,금융가 인사이드 고객 보험료로 지배구조 다지기 다시 도마 오른 삼성생명법,"[금융가, 인사이드, 고객, 보험료, 로, 지배, 구조, 다지기, 다시, 도마, 오...","[금융가, 인사이드, 고객, 보험료, 로, 지배, 구조, 다지기, 다시, 도마, 오...",1,42.03% 확률로 긍정 리뷰입니다.\n
52,삼성 아이폰에 낸드플래시 공급 적자 경쟁사 타격 불가피,"[삼성, 아이폰, 에, 낸드, 플래시, 공급, 적자, 경쟁사, 타격, 불가피]","[삼성, 아이폰, 낸드, 플래시, 공급, 적자, 경쟁사, 타격, 불가피]",1,52.98% 확률로 긍정 리뷰입니다.\n
54,삼성그룹 임원 상여금 십시일반으로 헌혈버스 지원,"[삼성그룹, 임원, 상여금, 십, 시, 일반, 으로, 헌혈, 버스, 지원]","[삼성그룹, 임원, 상여금, 십, 시, 일반, 헌혈, 버스, 지원]",1,53.43% 확률로 긍정 리뷰입니다.\n


In [84]:
news_df[news_df['predict'] == 2]

,title,token,remove_stopwords,predict,prob
